In [1]:
def load_fasta():
    from pathlib import Path
    diccionario_secuencias = {}
    base_path = Path("/home/alumno15/SB/practica1/secuencias").rglob('**/*.fasta') #utilizo .rglob para que busque recursivamente, en todas las carpetas de la carpeta madre, y con la expresión regular que acabe en .fasta
    for file in base_path: #hago un bucle para que pase por cada archivo acabado en .fasta que se encuentre
        file_stream = open(file, encoding="UTF-8") #Abrimos el flujo de lectura
        index = 0
        for line in file_stream:
            line = line.strip() #utilizo la función strip para quitar espacios que pudieran haber al principio o al final
            if index == 0: #el indice lo utilizo para distinguir la primera linea de las demas, esta comprobracion solo se realizara en la primera linea
                if line.startswith(">"): #si la primera linea empieza con > asumimos que es un fasta
                    nombre = line[1:] #y guardaremos esa linea en la variable nombre, quitando el primer caracter que es >
                    index += 1
                else: #si la primera linea no es > asumimos que no es un fasta y no hacemos nada mas con ese fichero, ya que no es un fasta
                    file_stream.close()
                    print("El archivo: ", file, "no es un FASTA correcto.") 
                    break
            else: #si el index no es 0 es que ya no estamos en la primera linea
                secuencia = line #y asignamos las lineas a secuencia
                diccionario_secuencias[nombre] = secuencia #metemos al diccionario secuencia como clave el nombre (primera linea) y como valor la secuencia
        file_stream.close() #cerramos el archivo
    return diccionario_secuencias



class Gen:
    nombre_completo = None
    secuencia = None
    nombresplit = None
    nombre_gen = None
    strand = None
    marco_lectura = None
    As = None
    Ts = None
    Gs = None
    Cs = None
    GC_porcentaje = None
    
    def __init__(self, nombre_completo, secuencia): #para definir instancias de esta clase voy a utilizar el diccionario, cogiendo la clave como nombre_completo y el valor como secuencia
        self.nombre_completo = nombre_completo
        self.secuencia = secuencia
        self.nombresplit = self.nombre_completo.split("|") #separo el nombre por barras verticales para luego usar esa informacion de la cabecera procesada
        self.nombre_gen = self.nombresplit[0] #la primera parte de la cabecera es el nombre del gen
        self.strand = self.nombresplit[1] #la segunda parte de la cabecera es la strand
        self.marco_lectura = self.nombresplit[2] #la tercera parte de la cabecera es el marco de lectura
        self.As = self.secuencia.count("A") #tambien meto en campos el numero de nucleotidos
        self.Ts = self.secuencia.count("T")
        self.Gs = self.secuencia.count("G")
        self.Cs = self.secuencia.count("C")
        self.GC_porcentaje = round((((self.Cs+ self.Gs)/len(self.secuencia))*100), 2) #y tambien el %GC

    def __str__(self):
        return self.nombre_completo
    
    def __repr__(self):
        return self.nombre_completo
    
    def forwardreverse(self):
        if self.strand == "F":
            secuenciaforward = self.secuencia 
            return secuenciaforward
        elif self.strand == "R":
            secuenciareverse = (self.secuencia)[::-1] #iterando sobre el string le digo que lo lea al revés
            return secuenciareverse
    
    def transcripcion(self):
        secuencia_arn = self.forwardreverse().replace("T", "U") #aquí he usado la función replace para cambiar T por U y lo hago llamando antes a la función anterior para ya transcribir sobre forward o reverse
        return secuencia_arn
        


class Codon:
    id = None
    codon = None
    aminoacid = None
    letter = None
    fullname = None
    
    def __init__(self, codon, aminoacid, letter, fullname): #esta clase la he creado poniendo las columnas del csv como campos, no tiene más
        self.id = id
        self.codon = codon
        self.aminoacid = aminoacid
        self.letter = letter
        self.fullname = fullname
    
    def __str__(self):
        return self.aminoacid
    
    def __repr__(self):
        return self.aminoacid



def load_csv_codonaa(file):
    from pathlib import Path
    import csv
    diccionario_codones = {}
    base_path = Path("/home/alumno15/SB/practica1")
    out_path = base_path.joinpath(file)
    with open(out_path, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=",") #Usamos el método DictReader para que use la cabecera del CSV como claves del diccionario
        index = 1
        codones = []
        for row in csv_reader:
            diccionario_fila = {}
            codonclase = Codon(row["codon"], row["aminoacid"], row["letter"], row["fullname"]) #me he ayudado de la clase para crear este diccionario
            diccionario_fila["codon"] = codonclase.codon
            diccionario_fila["aminoacid"] = codonclase.aminoacid
            diccionario_fila["letter"] = codonclase.letter
            diccionario_fila["fullname"] = codonclase.fullname
            diccionario_codones[index] = diccionario_fila
            codones.append(codonclase) #tambien meto en una lista las instancias que he creado de la clase codon para luego utilizarlas
            index += 1
    return diccionario_codones, codones



def opcion1(diccionario_secuencias):
    for gen in diccionario_secuencias:
            gen_opc1 = Gen(gen, diccionario_secuencias[gen])
            print(gen_opc1.nombre_gen, "; STRAND: ", gen_opc1.strand, "; MARCO LECTURA: ", gen_opc1.marco_lectura)
            print("\t" + "Longitud: ", len(gen_opc1.secuencia),"; ",
            "As :", gen_opc1.As,"; ",
            "Cs :", gen_opc1.Cs,"; ",
            "Ts :", gen_opc1.Ts,"; ",
            "Gs :", gen_opc1.Gs,"; ",
            "%GC :", gen_opc1.GC_porcentaje, "\n") #en esta opcion voy haciendo el print llamando a los campos de la clase Gen, y creo las instancias a partir del diccionario
    pass



def listar_ORF(secuencia_arn, gen):
    lista_secuencia_arn = list(secuencia_arn) #hago una lista de la secuencia de arn que ya está en forward o reverse
    lista_orf= [] #creo otra lista donde voy a meter todos los ORFs, es decir, será una lista de listas
    index = 0
    while index <= len(lista_secuencia_arn) - 3: #con el index controlo que no se me salga de rango y corte antes
        codon = lista_secuencia_arn[index:(index + 3)]
        if (codon == ["A", "U", "G"]):
            lista_codones = []
            if gen.marco_lectura == "1":
                lista_codones.append(codon)
                indexstop = index + 3
            elif gen.marco_lectura == "2": #aquí según sea el marco de lectura 1, 2 o 3 voy a ir sumando a los index a la hora de coger los nucleotidos de la secuencia
                codon = lista_secuencia_arn[index+1:(index + 4)]
                lista_codones.append(codon)
                indexstop = index + 3
            elif gen.marco_lectura == "3":
                codon = lista_secuencia_arn[index+2:(index + 5)]
                lista_codones.append(codon)
                indexstop = index + 3
            while indexstop <= len (lista_secuencia_arn) - 3:
                if gen.marco_lectura == "1":
                    codonstop = lista_secuencia_arn[indexstop: (indexstop + 3)] #y voy a hacer lo mismo que he hecho antes, pero con el codon de stop
                    lista_codones.append(codonstop)
                    if (codonstop == ["U", "A", "G"] or codonstop == ["U", "A", "A"] or codonstop == ["U", "G", "A"]):
                        lista_orf.append(lista_codones[:-1])
                        break
                elif gen.marco_lectura == "2":
                    codonstop = lista_secuencia_arn[indexstop+1: (indexstop + 4)]
                    lista_codones.append(codonstop)
                    if (codonstop == ["U", "A", "G"] or codonstop == ["U", "A", "A"] or codonstop == ["U", "G", "A"]):
                        lista_orf.append(lista_codones[:-1])
                        break
                elif gen.marco_lectura == "3":
                    codonstop = lista_secuencia_arn[indexstop+2: (indexstop + 5)]
                    lista_codones.append(codonstop)
                    if (codonstop == ["U", "A", "G"] or codonstop == ["U", "A", "A"] or codonstop == ["U", "G", "A"]):
                        lista_orf.append(lista_codones[:-1]) #cuando meto la lista de codones a la lista de orfs le quito el último porque el codon de stop no se añade en el ORF
                        break
                indexstop += 3 #controlando el indice del bucle que busca el STOP
        index += 1
    for orf in lista_orf:
        orf.insert(0, len(orf)) #a cada ORF le he añadido antes cuanto mide porque los voy a ordenar de menor a mayor numero de codones para que quede más visual
    lista_orf_sort = lista_orf.copy() #hago un copy de la lista anterior porque podría ser útil no cambiarla
    lista_orf_sort.sort(key=lambda x:x[0]) #Usamos la expresión lambda para acceder al elemento 1 de la lista, es decir, vamos a ordenar las listas segun su primer elemento
    resultado = []
    index = 0
    for i in lista_orf_sort:
        As = 0
        Cs = 0
        Us = 0
        Gs = 0
        for j in i[1:]: #aqui cuento lo que hay dentro de la lista quitando el primer elemento que era el del lenght
            As += j.count("A")
            Cs += j.count("C")
            Gs += j.count("G")
            Us += j.count("U")
        index +=1
        resultado.append([index, len(i[1:]), As, Cs, Gs, Us]) #la lista resultado va a tener los elementos que luego voy a poner en el print, porque no quería que el print estuviera dentro de la función, ya que si luego la utilizo para otra cosa me saca prints que no quiero
    return lista_orf_sort, resultado #voy a imprimir dos resultados, que iterando sobre la funcion luego puedes seleccionar
#el primero es la lista de orf ordenada de menos codones a más, que usaré después
#el segundo es el resultado con los elementos que quiero poner en el print



def traduccion(secuencia_arn, gen, numero_ORF): #esta funcion coge la secuencia de arn y el gen, que se le pasan a la funcion de listar ORF ya que la meto dentro, y también coge el número_ORF que es el número de input del ORF que queremos traducir
    resultadocsv = load_csv_codonaa("cod_amino.csv")
    codones = resultadocsv[1] #vamos a acceder al segundo return, que era la lista con todas las instancias creadas de la clase Codon
    listado_ORF = listar_ORF(secuencia_arn, gen)
    lista_orf_sort = listado_ORF[0] #accedemos a la lista de orfs ordenados de menos a mas codones
    traduccion = []
    if numero_ORF <= len(lista_orf_sort) and numero_ORF > 0: #esto es de control de errores por si el numero introducido está mal
        for j in lista_orf_sort[numero_ORF-1][1:]: #vamos a iterar sobre el orf que queramos, que estará en la posición del numero seleccionado - 1 (ya que las listas empiezan a contar por 0), y luego le quitamos el primer elemento de la lista que hemos dicho antes que era el lenght
            jstring = "".join(j) #pasamos la lista a string
            for codonclase in codones:
                if jstring == codonclase.codon: #si la string es igual que un campo de una instancia de la clase codon
                    traduccion.append(codonclase.letter) #se añadirá de esa instancia en concreto su campo letter, y así se consigue la traduccion
        traduccionstring = "".join(traduccion) #pasamos la lista traducciona  string
        return traduccionstring #y devolvemos el string de traduccion
    else:
        print("\nIntroduce un número de los que aparecen en la lista de ORFs.")
        return
    

                
                
def opcion2(diccionario_secuencias, diccionario_codones, secuencia):
    genes = []
    index = 0
    for gen in diccionario_secuencias:
        gen_opc2 = Gen(gen, diccionario_secuencias[gen])
        genes.append(gen_opc2) #guardamos en la lista genes todas las instancias de la clase Gen creadas a partir del diccionario de secuencias
        if secuencia == gen_opc2.nombre_gen: #y vamos a imprimir lo mismo que en la opcion 1 pero solo con el gen cuyo campo de nombre sea igual que la secuencia imput 
            print(gen_opc2.nombre_gen, "; STRAND: ", gen_opc2.strand, "; MARCO LECTURA: ", gen_opc2.marco_lectura)
            print("\t" + "Longitud: ", len(gen_opc2.secuencia),"; ",
            "As :", gen_opc2.As,"; ",
            "Cs :", gen_opc2.Cs,"; ",
            "Ts :", gen_opc2.Ts,"; ",
            "Gs :", gen_opc2.Gs,"; ",
            "%GC :", gen_opc2.GC_porcentaje, "\n")
            break
        elif index < len(diccionario_secuencias)-1: #esto es para controlar que no se corte el bucle si no lo encuentra la primera vez
            index += 1
            continue
        else: #y esto en el caso de que se haya introducido algo incorrecto
            print("\nLa secuencia no está en la lista, escriba una de la lista.",
                  "\nPara consultar las secuencias que aparecen en la lista seleccione la opción 1.")
            return
    print("\t" + "ORFs: ")
    secuencia_arn = gen_opc2.transcripcion() #del gen escogido llamamos a su campo transcripcion
    #for gen in genes:
    listado_ORF = listar_ORF(secuencia_arn, gen_opc2) #y a la funcion de listar_ORF
    #print(listado_ORF[1])
    for i in listado_ORF[1]: #y con la lista de resultado vamos a ir metiendo en el print los distintos items de la lista resultado
        print("\t" + "\t", i[0], ". ", i[1], " codones. As: ", i[2], ", Cs: ", i[3], ", Gs: ", i[4], ", Us: ", i[5])
    while True:
        print("\nOPCIONES:")
        print("\t" + "1. Traducir ORF")
        print("\t" + "2. Volver atrás")
        opcion = input("\nIndique la opción que desea ejecutar (1 o 2): ")
        if opcion == "1":
            numero_ORF_input = input("\nIndique el número del RNA que desea traducir: ")
            try: #aqui hay un control de errores en el caso de que hayamos introducido una palabra y no un número
                numero_ORF = int(numero_ORF_input)
            except:
                print("\nDebes introducir un número de los que aparecen en la lista de ORFs.")
                continue
            traduccionstring = traduccion(secuencia_arn, gen_opc2, numero_ORF) #y aqui llamamos a la funcion traduccion
            if traduccionstring == None: #aqui en el caso de que se haya metido algo mal, para que no nos de un resultado de None, sino que nos devuelva a la pantalla anterior
                continue
            else:
                print("\nEl péptido resultado es: ", traduccionstring) #aqui el resultado en el caso en el que todas las condiciones se cumplan
        elif opcion == "2":
            break
        else:
            print("La opción indicada no está disponible. Pruebe con la siguientes:\n")
    else:
        print("La secuencia no aparece, introduce una secuencia válida")
    pass



def main():
    diccionario_secuencias = load_fasta()
    diccionario_codones = load_csv_codonaa("cod_amino.csv")
    while True:
        print("\n\n1. Listar secuencias de DNA")
        print("2. Detalle de una secuencia")
        print("3. Salir del programa")
        opcion = input("\nIndique la opción que desea ejecutar (1, 2 o 3): ")
        if opcion == "1":
            #Imprimir diccionario_secuencias en el formato indicado
            opcion1(diccionario_secuencias)
        elif opcion == "2":
            secuencia = input("¿Qué secuencia quieres saber en detalle? ") #aqui la secuencia imput que le pasaremos a la opcion 2
            opcion2(diccionario_secuencias, diccionario_codones, secuencia)
        elif opcion == "3":
            break
        else:
            #IMPORTANTE. Avisar al usuario cuando la entrada no es correcta
            print("La opción indicada no está disponible. Pruebe con la siguientes:\n")
    print("El programa ha finalizado correctamente.")

if __name__ == "__main__":
    main()

El archivo:  /home/alumno15/SB/practica1/secuencias/secuencias1/seq13.fasta no es un FASTA correcto.
El archivo:  /home/alumno15/SB/practica1/secuencias/secuencias1/.ipynb_checkpoints/seq13-checkpoint.fasta no es un FASTA correcto.
El archivo:  /home/alumno15/SB/practica1/secuencias/secuencias3/seq12.fasta no es un FASTA correcto.
El archivo:  /home/alumno15/SB/practica1/secuencias/secuencias3/.ipynb_checkpoints/seq12-checkpoint.fasta no es un FASTA correcto.


1. Listar secuencias de DNA
2. Detalle de una secuencia
3. Salir del programa



Indique la opción que desea ejecutar (1, 2 o 3):  2
¿Qué secuencia quieres saber en detalle?  sfsgd



La secuencia no está en la lista, escriba una de la lista. 
Para consultar las secuencias que aparecen en la lista seleccione la opción 1.


1. Listar secuencias de DNA
2. Detalle de una secuencia
3. Salir del programa



Indique la opción que desea ejecutar (1, 2 o 3):  1


gen1 ; STRAND:  R ; MARCO LECTURA:  1
	Longitud:  509 ;  As : 138 ;  Cs : 129 ;  Ts : 123 ;  Gs : 119 ;  %GC : 48.72 

gen2 ; STRAND:  F ; MARCO LECTURA:  1
	Longitud:  525 ;  As : 127 ;  Cs : 138 ;  Ts : 115 ;  Gs : 145 ;  %GC : 53.9 

gen3 ; STRAND:  F ; MARCO LECTURA:  1
	Longitud:  565 ;  As : 142 ;  Cs : 126 ;  Ts : 145 ;  Gs : 152 ;  %GC : 49.2 

gen4 ; STRAND:  F ; MARCO LECTURA:  1
	Longitud:  510 ;  As : 119 ;  Cs : 137 ;  Ts : 127 ;  Gs : 127 ;  %GC : 51.76 

gen5 ; STRAND:  F ; MARCO LECTURA:  2
	Longitud:  595 ;  As : 137 ;  Cs : 168 ;  Ts : 147 ;  Gs : 143 ;  %GC : 52.27 

gen6 ; STRAND:  F ; MARCO LECTURA:  2
	Longitud:  573 ;  As : 132 ;  Cs : 138 ;  Ts : 159 ;  Gs : 144 ;  %GC : 49.21 

gen7 ; STRAND:  R ; MARCO LECTURA:  2
	Longitud:  570 ;  As : 146 ;  Cs : 153 ;  Ts : 121 ;  Gs : 150 ;  %GC : 53.16 

gen8 ; STRAND:  R ; MARCO LECTURA:  2
	Longitud:  594 ;  As : 141 ;  Cs : 156 ;  Ts : 159 ;  Gs : 138 ;  %GC : 49.49 

gen9 ; STRAND:  F ; MARCO LECTURA:  3
	Longitud:  

...


El péptido resultado es:  OAGFNTTKDFRILDCNTYKILTP

OPCIONES:
	1. Traducir ORF
	2. Volver atrás



Indique la opción que desea ejecutar (1 o 2):  2




1. Listar secuencias de DNA
2. Detalle de una secuencia
3. Salir del programa



Indique la opción que desea ejecutar (1, 2 o 3):  3


El programa ha finalizado correctamente.
